## Validation 和 Cross Validation
### 一、传统分割的缺点
![IMAGE](https://farm2.staticflickr.com/1724/42797474242_543ed41c58_o.png)
因为测试数据集的 label 是已知的，我们所有参数的调节都是围绕 test 数据集。这样会导致最终的结果**针对测试数据集**过度拟合。
> 我们的目标是真实环境的数据，而不仅仅是测试数据集。


### 二、验证数据集的引入
![IMAGE](https://farm2.staticflickr.com/1724/27977206767_0d9a58abc8_o.png)

* 用验证数据集代替原来的测试数据：用来调整超参数
* 测试数据集模拟真实环境中的位置数据

问题：如果验证数据集中有极端的数据，我们的模型可能不准确。

### 三、交叉验证
![IMAGE](https://farm2.staticflickr.com/1821/42128260574_498872a0f2_o.png)

这里把 train set 分为 ABC 三份，分别作验证数据集。

因为我们取 k 个平均，会减弱由于一份据集中极端数据造成的偏差。

In [2]:
import numpy as np
from sklearn import datasets

In [3]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 测试train_test_split

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [5]:
from sklearn.neighbors import KNeighborsClassifier

best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6): # knn 中的距离指数
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 3
Best P = 4
Best Score = 0.986091794159


### 使用交叉验证

In [6]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([ 0.98895028,  0.97777778,  0.96629213])

sklearn 默认把训练数据集分成了**3份**，进行交叉验证。但究竟如何分组，sklearn 封装的比价复杂。

In [7]:
best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # ❤️ 添加cross validation
        scores = cross_val_score(knn_clf, X_train, y_train)
        # 取平均值
        score = np.mean(scores)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 2
Best P = 2
Best Score = 0.982359987401


交叉验证的结果：0.982359987401

train test split 验证的结果：0.986091794159

为什么交叉验证的结果低呢？

> 因为交叉验证通常不会过拟合数据，所以分数相对应比较低。

但是最终的结果不是0.982，通过交叉验证，我们只是得到最优的超参数。

In [8]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.98052851182197498

这个才是最终的分类准确度。
> 注意这里使用的 X_test, 和 y_test 是第一次引入，对模型来说，是完全陌生的。所以这个准确率是更值得相信的

### 回顾网格搜索

In [9]:
from sklearn.model_selection import GridSearchCV
# 这里的 CV 就是 CrossValidation
param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

* 3 folds 就是把训练集交叉验证3份
* 45 个 candidate 
* 所以总共有135次训练

In [10]:
grid_search.best_score_

0.98237476808905377

之前网格搜索得到的结果和我们交叉验证

In [11]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [12]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.98052851182197498

### cv参数 训练数据集分成的份数

In [13]:
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([ 0.99543379,  0.96803653,  0.98148148,  0.96261682,  0.97619048])

In [14]:
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)

![IMAGE](https://farm2.staticflickr.com/1742/41044080180_b8e73aff4b_o.png)

![IMAGE](https://farm2.staticflickr.com/1734/42137036174_9c7e652490_o.png)
这个计算量太大了，通常实际上不会用，但是论文的严谨性，会使用这种方法